In [1]:
import os
import scipy.io as sio
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

사용 데이터 경로  
C:\Users\ChoiSeongHyeon\Desktop\Dataset\UOS\BearingType_DeepGrooveBall\SamplingRate_16000

In [2]:
import scipy.io

# MATLAB 파일 경로
file_path = r"C:\Users\ChoiSeongHyeon\Desktop\Dataset\UOS\BearingType_DeepGrooveBall\SamplingRate_16000\RotatingSpeed_1000\H_B_16_6204_1000.mat"

def print_mat_contents(file_path):
    try:
        # MATLAB 파일 로드
        mat_contents = scipy.io.loadmat(file_path)
        
        # MATLAB 파일 내용 출력
        print("MATLAB 파일의 키 목록:")
        for key in mat_contents.keys():
            # 내부 데이터 구조를 확인하기 위해 데이터의 타입과 샘플 출력
            print(f"- {key}: {type(mat_contents[key])}")
            if isinstance(mat_contents[key], (list, dict, tuple)):
                print(f"  샘플 데이터: {mat_contents[key][:2]}")
            elif hasattr(mat_contents[key], 'shape'):
                print(f"  데이터 형상(shape): {mat_contents[key].shape}")
            else:
                print(f"  값: {mat_contents[key]}")

    except Exception as e:
        print(f"MATLAB 파일을 읽는 중 오류가 발생했습니다: {e}")

# 함수 실행
print_mat_contents(file_path)


MATLAB 파일의 키 목록:
- __header__: <class 'bytes'>
  값: b'MATLAB 5.0 MAT-file, Platform: PCWIN64, Created on: Wed Jun  5 10:40:04 2024'
- __version__: <class 'str'>
  값: 1.0
- __globals__: <class 'list'>
  샘플 데이터: []
- Data: <class 'numpy.ndarray'>
  데이터 형상(shape): (1280000, 1)
- STFTFreq: <class 'numpy.ndarray'>
  데이터 형상(shape): (128, 1)
- STFTTime: <class 'numpy.ndarray'>
  데이터 형상(shape): (1, 128)
- Spectrogram: <class 'numpy.ndarray'>
  데이터 형상(shape): (78, 128, 128)


# Cell 1. 데이터 Sampling  
- Origin Data -> Test, Validation, Train (6:2:2)로 구분
- SAMPLE_SIZE : 2,048, SHIFT_SIZE : 1,024

Origin 파일의 길이 : 1280,000  
Shift Size : 1024  
따라서, 1,250개의 샘플 생성되어야 함.

In [ ]:
# pip install scipy numpy tqdm

In [5]:
import os
import numpy as np
import scipy.io
import pandas as pd
from tqdm import tqdm

# 기본 경로 설정
BASE_DIR = r"C:\Users\ChoiSeongHyeon\Desktop\WinningT\Winning Ticket\MyWinningTicket\Dataset\UOS"
FAULT_TYPES = ["H", "B", "IR", "OR"]
SPLIT_RATIOS = (0.6, 0.2, 0.2)  # Train: 60%, Val: 20%, Test: 20%
SAMPLE_SIZE = 2048
SHIFT_SIZE = 1024

def load_mat_file(file_path):
    """ .mat 파일에서 'Data' 키를 가진 데이터를 불러옴 """
    mat_data = scipy.io.loadmat(file_path)
    if 'Data' in mat_data:
        return mat_data['Data'].flatten()  # 1D 배열로 변환
    else:
        raise KeyError(f"'Data' key not found in {file_path}")

def split_data(data, split_ratios):
    """ 데이터를 Train, Validation, Test로 분할 """
    total_len = len(data)
    train_end = int(total_len * split_ratios[0])
    val_end = train_end + int(total_len * split_ratios[1])

    train_data = data[:train_end]
    val_data = data[train_end:val_end]
    test_data = data[val_end:]

    return train_data, val_data, test_data

def save_csv_file(file_path, data):
    """ 데이터를 CSV 파일로 저장 (단일 열 형태) """
    df = pd.DataFrame(data)  # ✅ 단일 열(column) 형태로 변환
    df.to_csv(file_path, index=False, header=False)  # ✅ 열 이름 없이 저장

def create_samples(data, save_dir, base_filename):
    """ 데이터를 2048 크기로 1024씩 이동하며 개별 CSV 파일로 저장 """
    os.makedirs(save_dir, exist_ok=True)
    num_samples = int((len(data) / SHIFT_SIZE) - 1)

    for i in range(num_samples):
        start = i * SHIFT_SIZE
        end = start + SAMPLE_SIZE
        if end > len(data):  
            break

        sample = data[start:end]
        sample_filename = os.path.join(save_dir, f"{base_filename}_{i+1}.csv")
        save_csv_file(sample_filename, sample)

def process_fault_type(fault_type):
    """ 각 Fault Type 폴더의 파일을 처리 """
    fault_dir = os.path.join(BASE_DIR, fault_type, "Origin")
    split_dir = os.path.join(BASE_DIR, fault_type, "Train_Val_Test_Split")
    samples_dir = os.path.join(BASE_DIR, fault_type, "Samples")

    os.makedirs(split_dir, exist_ok=True)
    os.makedirs(samples_dir, exist_ok=True)

    # Origin 폴더 내 모든 .mat 파일 처리
    for file in tqdm(os.listdir(fault_dir), desc=f"Processing {fault_type}"):
        if not file.endswith(".mat"):
            continue

        file_path = os.path.join(fault_dir, file)
        base_name = file.replace(".mat", "")

        try:
            # .mat 파일 로드
            data = load_mat_file(file_path)
            
            # Train, Validation, Test 데이터 분할
            train_data, val_data, test_data = split_data(data, SPLIT_RATIOS)

            # Train, Validation, Test 데이터를 CSV 파일로 저장
            save_csv_file(os.path.join(split_dir, f"{base_name}_Train.csv"), train_data)
            save_csv_file(os.path.join(split_dir, f"{base_name}_Validation.csv"), val_data)
            save_csv_file(os.path.join(split_dir, f"{base_name}_Test.csv"), test_data)

            # Train, Validation, Test 데이터에서 개별 샘플 생성 및 저장
            create_samples(train_data, samples_dir, f"{base_name}_Train")
            create_samples(val_data, samples_dir, f"{base_name}_Validation")
            create_samples(test_data, samples_dir, f"{base_name}_Test")

        except Exception as e:
            print(f"Error processing {file}: {e}")

# 모든 Fault Type에 대해 실행
for fault in FAULT_TYPES:
    process_fault_type(fault)

print("✅ 모든 데이터 처리가 완료되었습니다!")


Processing OR: 100%|██████████| 6/6 [00:48<00:00,  8.05s/it]

✅ 모든 데이터 처리가 완료되었습니다!


- Rotating Speed에 상관없이 Train Data로 섞기!

# Cell 2. Labelling  
H : 0, IR : 1, OR : 2, B : 3  

# Cell 3. WDCNN